In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [2]:
def preprocess_text(text_series):
    return (text_series
            .str.lower()
            .replace(r'[^a-z0-9\s]', ' ', regex=True)
            .str.strip()
            .str.split()
            .apply(lambda tokens: ' '.join(tokens)))

In [3]:
def preprocess_df(df):
    df = df.drop(columns=['EM'])
    
    df['Description'] = preprocess_text(df['Description'])
    df['EN'] = preprocess_text(df['EN'])
    df['combined_text'] = df['Description'] + ' ' + df['EN']
    
    train_df, test_df = train_test_split(
        df, test_size=0.2, random_state=42, stratify=df['Composition strategy']
    )
    train_df, val_df = train_test_split(
        train_df, test_size=0.2, random_state=42, stratify=train_df['Composition strategy']
    )
    
    return train_df, val_df, test_df

In [5]:
def load_glove_embeddings(filepath):
    glove_dict = {}
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            glove_dict[word] = vector
    return glove_dict

def sentence_to_glove(sentence, glove_dict, embedding_dim=100):
    tokens = sentence.split()
    valid_vectors = [glove_dict[token] for token in tokens if token in glove_dict]
    if len(valid_vectors) == 0:
        return np.zeros(embedding_dim, dtype='float32')

    return np.mean(valid_vectors, axis=0)

In [6]:
class ELCoGloveDataset(Dataset):
    def __init__(self, df, glove_dict, embedding_dim=100):
        self.sentences = df['combined_text'].tolist()
        self.labels = df['Composition strategy'].tolist()
        self.glove_dict = glove_dict
        self.embedding_dim = embedding_dim
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        embedding = sentence_to_glove(sentence, self.glove_dict, self.embedding_dim)
        embedding = torch.tensor(embedding, dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return embedding, label

In [20]:
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLPClassifier, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
        
    def forward(self, x):
        return self.net(x)


In [7]:
glove_path = '../../embeddings/glove.6B.100d.txt'
glove_embeddings = load_glove_embeddings(glove_path)
embedding_dim = 100

In [9]:
ELCo_df = pd.read_csv('../../data/ELCo.csv')
label_encoder = LabelEncoder()
ELCo_df['Composition strategy'] = label_encoder.fit_transform(ELCo_df['Composition strategy'])

In [13]:
batch_size = 32
train_df, val_df, test_df = preprocess_df(ELCo_df)
train_dataset = ELCoGloveDataset(train_df, glove_embeddings, embedding_dim)
val_dataset   = ELCoGloveDataset(val_df, glove_embeddings, embedding_dim)
test_dataset  = ELCoGloveDataset(test_df, glove_embeddings, embedding_dim)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [21]:
input_dim = embedding_dim  
hidden_dim = 256          
output_dim = len(label_encoder.classes_) 
model = MLPClassifier(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
model.to(device)

MLPClassifier(
  (net): Sequential(
    (0): Linear(in_features=100, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=256, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=256, out_features=256, bias=True)
    (7): ReLU()
    (8): Linear(in_features=256, out_features=5, bias=True)
  )
)

In [22]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for embeddings, labels in train_loader:
        embeddings = embeddings.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(embeddings)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    
    # Validation step
    model.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for embeddings, labels in val_loader:
            embeddings = embeddings.to(device)
            labels = labels.to(device)
            outputs = model(embeddings)
            _, preds = torch.max(outputs, dim=1)
            total_correct += (preds == labels).sum().item()
            total_samples += labels.size(0)
    val_accuracy = total_correct / total_samples
    print(f"Epoch [{epoch+1}/{num_epochs}] | Train Loss: {avg_loss:.4f} | Val Accuracy: {val_accuracy:.4f}")

Epoch [1/100] | Train Loss: 1.2804 | Val Accuracy: 0.4528
Epoch [2/100] | Train Loss: 1.1849 | Val Accuracy: 0.5698
Epoch [3/100] | Train Loss: 1.1404 | Val Accuracy: 0.5434
Epoch [4/100] | Train Loss: 1.0665 | Val Accuracy: 0.5660
Epoch [5/100] | Train Loss: 1.0252 | Val Accuracy: 0.5736
Epoch [6/100] | Train Loss: 0.9416 | Val Accuracy: 0.5698
Epoch [7/100] | Train Loss: 0.8832 | Val Accuracy: 0.5509
Epoch [8/100] | Train Loss: 0.8275 | Val Accuracy: 0.5811
Epoch [9/100] | Train Loss: 0.7849 | Val Accuracy: 0.6075
Epoch [10/100] | Train Loss: 0.7177 | Val Accuracy: 0.6075
Epoch [11/100] | Train Loss: 0.6821 | Val Accuracy: 0.6113
Epoch [12/100] | Train Loss: 0.6509 | Val Accuracy: 0.6113
Epoch [13/100] | Train Loss: 0.5964 | Val Accuracy: 0.5698
Epoch [14/100] | Train Loss: 0.5596 | Val Accuracy: 0.5925
Epoch [15/100] | Train Loss: 0.5305 | Val Accuracy: 0.6151
Epoch [16/100] | Train Loss: 0.4481 | Val Accuracy: 0.6075
Epoch [17/100] | Train Loss: 0.4497 | Val Accuracy: 0.6264
Epoch 

In [23]:
model.eval()
total_correct = 0
total_samples = 0
with torch.no_grad():
    for embeddings, labels in test_loader:
        embeddings = embeddings.to(device)
        labels = labels.to(device)
        outputs = model(embeddings)
        _, preds = torch.max(outputs, dim=1)
        total_correct += (preds == labels).sum().item()
        total_samples += labels.size(0)
test_accuracy = total_correct / total_samples
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.6073
